In [2]:
from src.core import *
from src import DSL
import inspect
import typing

import asyncio

from poke_env.player.random_player import RandomPlayer
from poke_env.player.utils import cross_evaluate
from poke_env.player_configuration import PlayerConfiguration
from poke_env.server_configuration import LocalhostServerConfiguration
from poke_env.utils import to_id_str
from tabulate import tabulate

import trueskill
import copy
from trueskill import Rating, rate_1vs1, quality_1vs1
from tqdm.notebook import tqdm
import itertools
import numpy as np
import pandas as pd
from pprint import pprint


import logging
# from aiologger.handlers.files import AsyncFileHandler
# from aiologger.handlers.files import AsyncFileHandler
from tempfile import NamedTemporaryFile

from pathlib import Path
import pickle
import dill
import gc
import math

In [3]:
class PlayerWrapper(Player):
    def __init__(self, *args, **kwargs):
        super().__init__(
            player_configuration=PlayerConfiguration(get_node_id(), None),
            battle_format="gen7randombattle",
            server_configuration=LocalhostServerConfiguration,
            max_concurrent_battles=100000000,
        )
        self.script = None
        
#         self.log_file = NamedTemporaryFile()
#         self.patch_logger()
    
#     def patch_logger(self):
#         self.logger.removeHandler(self.logger.handlers[0])
#         self.logger.addHandler(logging.FileHandler(self.log_file.name))
#         self.logger.
        
    def choose_move(self, battle):
        try:
            move = self.script.choose_move(battle)
            return self.create_order(move)
        except Exception as e:
            print(e)
            return self.choose_random_move(battle)

In [19]:
chunk_size = 512
shared_players = [PlayerWrapper() for _ in range(chunk_size)]
player_lookup = {player.username: player for player in shared_players}

In [20]:
async def evaluate_population(population, num_games, verbose=True):
    for script in population:
        script.rating = Rating()
    
    num_players = min(chunk_size, len(population))
    players = shared_players[:num_players]
    num_chunks = int(math.ceil(len(population) / chunk_size))
    chunks = [population[i * chunk_size : (i + 1) * chunk_size] for i in range(num_chunks)]
    
    for chunk_idx, population in enumerate(chunks):
        print(f"chunk {chunk_idx + 1} / {num_chunks}")
        
        for i, script in enumerate(population):
            players[i].script = script
        
        for _ in tqdm(range(num_games), disable=not verbose):
            random.shuffle(players)
            awaitables = []
            for p1, p2 in zip(players[0::2], players[1::2]):
                send = p1.send_challenges(
                    opponent=to_id_str(p2.username),
                    n_challenges=1,
                    to_wait=p2.logged_in,
                )
                accept = p2.accept_challenges(
                    opponent=to_id_str(p1.username),
                    n_challenges=1,
                )
                awaitables.append(send)
                awaitables.append(accept)
            try:
                await asyncio.gather(*awaitables)
            except Exception as e:
                print(e)
                battles = sum((list(player.battles.values()) for player in players), [])
                return battles

        battles = sum((list(player.battles.values()) for player in players), [])
        for battle in battles:
            player = player_lookup[battle.player_username]
            oppo = player_lookup[battle._opponent_username]

            if battle.won:
                winner, loser = player, oppo
            else:
                winner, loser = oppo, player
            winner.script.rating, loser.script.rating = rate_1vs1(winner.script.rating, loser.script.rating)

    del players
    gc.collect()

In [21]:
def sort_population(population):
    return sorted(population, key=lambda p: p.rating.mu, reverse=True)

# def sort_population(population):
#     return sorted(population, key=lambda p: p.n_won_battles, reverse=True)

In [22]:
def get_elites(population, num_elites):
    return sort_population(population)[:num_elites]

In [23]:
def get_tournament_elites(population, tournament_size, num_elites):
    tournament = random.sample(population, tournament_size)
    return get_elites(tournament, num_elites)

In [24]:
def mutate(tree, filter_=None):
    if not filter_:
        filter_ = lambda x: type(x) is not str
    tree = copy.deepcopy(tree)
    candidates = anytree.search.findall(tree, filter_=filter_)
    if candidates:
        target = random.choice(candidates)
        target.children = []
        generate_tree(target)
    return tree

In [25]:
def crossover(left, right):
    def _is_valid_subtree_head(node):
#         return node.name in [RULE.IF_BLOCK]
        return True

    def _find_head_types(root):
        return [node.name for node in anytree.search.findall(root, filter_=_is_valid_subtree_head)]

    def _select_type_node(root, type_):
        candidates = anytree.search.findall(root, filter_=lambda node: node.name == type_)
        return random.choice(candidates)
    
    def _swap_children(left, right):
        left.children, right.children = right.children, left.children
        
    left = copy.deepcopy(left)
    right = copy.deepcopy(right)

    left_head_types = _find_head_types(left)
    right_head_types = _find_head_types(right)

    type_intersection = list(set(left_head_types) & set(right_head_types))
    if type_intersection:
        type_to_swap = random.choice(type_intersection)
        left_head = _select_type_node(left, type_to_swap)
        right_head = _select_type_node(right, type_to_swap)
        _swap_children(left_head, right_head)
    return left, right

In [26]:
def get_random_population(population_size):
    return [exec_tree(get_random_tree()) for _ in range(population_size)]

In [27]:
# def get_population_stats(population):
#     ratings = pd.Series([player.rating.mu for player in population])
#     return ratings.describe()

In [28]:
def save_population(population, fname):
    path = Path('temp_scripts').joinpath(fname)
    with open(path, 'wb') as f:
        dill.dump([player.tree for player in population], f)
            
def load_population(fname):
    path = Path('temp_scripts').joinpath(fname)
    with open(path, 'rb') as f:
        return [exec_tree(tree) for tree in dill.load(f)]

# Evolve

In [29]:
# sanity check
num_elites = 2
tournament_size = 4
population_cap = 24
epochs = 10
num_games = 5

In [30]:
# medium
num_elites = 2
tournament_size = 5
population_cap = 48
epochs = 32
num_games = 5

In [32]:
# large
num_elites = 3
tournament_size = 6
population_cap = 64
epochs = 48
num_games = 5

In [33]:
population = get_random_population(population_size=population_cap)
generations = []
for i in tqdm(range(epochs)):
    save_population(population, f"{i}.dill")
    await evaluate_population(population, num_games=num_games, verbose=False)
#     assign_trueskill(population)
    generations.append([copy.deepcopy(script) for script in population])
    population = sort_population(population)
    next_population = []
    next_population.extend(get_elites(population, num_elites))
    while len(next_population) < population_cap:
        left, right = get_tournament_elites(population, tournament_size, 2)
        children = crossover(left.tree, right.tree)
        for child_tree in children:
            child_tree = mutate(child_tree)
            next_population.append(exec_tree(child_tree))
    population = next_population[:population_cap]
    gc.collect()

chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
'int' object has no attribute 'damage_multiplier'
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
'int' object has no attribute 'damage_multiplier'
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
chunk 1 / 1
'int' object has no attribute 'damage_multiplier'
'int' object has no 

# Evaluate

In [26]:
generations = [load_population(f"{i}.dill") for i in range(32)]

In [37]:
all_scripts = sum(generations, [])
len(all_scripts)

3119

In [39]:
battles = await evaluate_population(all_scripts, num_games=10, verbose=True)

chunk 1 / 7


'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'

chunk 2 / 7


2020-04-14 00:24:06,699 - a1b4a299 - CRITICAL - An error occured while adding available moves. The following move was either unknown or not available for the active pokemon: outrage



chunk 3 / 7


'int' object has no attribute 'damage_multiplier'

chunk 4 / 7


...

2020-04-14 00:58:47,566 - 58a5ef62 - WARNING - Trying to login as 58a5ef62, showdown returned  58a5ef62@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,567 - f181a6c4 - WARNING - Trying to login as f181a6c4, showdown returned  f181a6c4@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,568 - 64af3fd7 - WARNING - Trying to login as 64af3fd7, showdown returned  64af3fd7@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,569 - 60036f65 - WARNING - Trying to login as 60036f65, showdown returned  60036f65@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,571 - 46351d28 - WARNING - Trying to login as 46351d28, showdown returned  46351d28@! - this might prevent future action

2020-04-14 00:58:47,780 - c7b5a90a - WARNING - Trying to login as c7b5a90a, showdown returned  c7b5a90a@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,783 - 76ab1520 - WARNING - Trying to login as 76ab1520, showdown returned  76ab1520@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,787 - a64a9f5b - WARNING - Trying to login as a64a9f5b, showdown returned  a64a9f5b@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,792 - f25bdfdd - WARNING - Trying to login as f25bdfdd, showdown returned  f25bdfdd@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,796 - 00e8d053 - WARNING - Trying to login as 00e8d053, showdown returned  00e8d053@! - this might prevent future action

2020-04-14 00:58:47,890 - ec289d7a - WARNING - Trying to login as ec289d7a, showdown returned  ec289d7a@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,891 - d6e20370 - WARNING - Trying to login as d6e20370, showdown returned  d6e20370@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,897 - 255ea901 - WARNING - Trying to login as 255ea901, showdown returned  255ea901@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,900 - 6f67bd21 - WARNING - Trying to login as 6f67bd21, showdown returned  6f67bd21@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:47,901 - 5b5704a3 - WARNING - Trying to login as 5b5704a3, showdown returned  5b5704a3@! - this might prevent future action

2020-04-14 00:58:48,000 - f1c8e1f2 - WARNING - Trying to login as f1c8e1f2, showdown returned  f1c8e1f2@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:48,002 - 37aae601 - WARNING - Trying to login as 37aae601, showdown returned  37aae601@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:48,003 - d5d47399 - WARNING - Trying to login as d5d47399, showdown returned  d5d47399@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:48,005 - 975f3182 - WARNING - Trying to login as 975f3182, showdown returned  975f3182@! - this might prevent future actions from this agent. Changing the agent's username might solve this problem.
2020-04-14 00:58:48,006 - a445f347 - WARNING - Trying to login as a445f347, showdown returned  a445f347@! - this might prevent future action

'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'
'int' object has no attribute 'damage_multiplier'



In [40]:
script_to_gen = {}
for i, gen in enumerate(generations):
    for script in gen:
        script_to_gen[script] = i

In [47]:
gen_ratings = [[] for _ in range(len(generations))]
for script in all_scripts:
    gen_idx = script_to_gen[script]
    gen_ratings[gen_idx].append(script.rating.mu)
gen_ratings[0].append(random.choice(gen_ratings[0]))
gen_ratings = np.asarray(gen_ratings)

In [43]:
xs = np.arange(len(gen_ratings))
ys_high = np.percentile(gen_ratings, 75, axis=-1)
ys_median = np.median(gen_ratings, axis=-1)
ys_low = np.percentile(gen_ratings, 25, axis=-1)

TypeError: can't multiply sequence by non-int of type 'float'

In [41]:
from bokeh.plotting import figure, output_file, show, reset_output, output_notebook

reset_output()
output_notebook()

p = figure(plot_width=800, plot_height=400, y_range=(15, 40))
p.line(xs, ys_high, line_width=2, alpha=0.3)
p.line(xs, ys_median, line_width=2)
p.line(xs, ys_low, line_width=2, alpha=0.3)

show(p)

Loading BokehJS ...

In [27]:
best_script = sort_population(all_scripts)[0]
print(best_script.rating)
print(best_script.raw_script)

trueskill.Rating(mu=41.847, sigma=3.465)

class Script_af4248aeb03c4a23b7bbfa3fa68cf72f(Script):
    def choose_move(self, battle: Battle):
    
        available_moves = battle.available_moves
        if not available_moves:
            return random.choice(battle.available_switches)
            
        dsl = DSL(battle)
        move_scores = []
        for move in battle.available_moves:
            score = 0
            if (not (dsl.is_sunny_and_move_is_not_water(move))):
                if (not (dsl.is_special_attacker_and_move_special(move))):
                    score += -2
            if (not (dsl.is_physical_attacker_and_move_physical(move))):
                if ((dsl.is_physical_attacker_and_move_physical(move) and dsl.is_super_effective(move))):
                    score += -1
                if (not (dsl.is_raining_and_move_is_not_fire(move))):
                    score += 2
                if ((dsl.is_sunny_and_move_is_fire(move) and dsl.is_sunny_and_move_is_fire(move))):


In [ ]:
unfinished = []
for p in population:
    for b in p.battles.values():
        if not b.finished:
            unfinished.append(b)

In [ ]:
lk = {p.username: p for p in population}